In [45]:
using RigidBodyDynamics
using StaticArrays
using MeshCat, MeshCatMechanisms, Blink
using Random
Random.seed!(42);

In [46]:
srcdir = dirname(pathof(RigidBodyDynamics))
urdf = joinpath(srcdir, "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)
state = MechanismState(mechanism)
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [47]:
body = findbody(mechanism, "lower_link")
point = Point3D(default_frame(body), 0., 0, -2)

Point3D in "after_elbow": [0.0, 0.0, -2.0]

In [48]:
# Create the visualizer
vis = MechanismVisualizer(mechanism, URDFVisuals(urdf))

# Render our target point attached to the robot as a sphere with radius 0.07
setelement!(vis, point, 0.07)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8708
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat/world/upper_link/lower_link/after_elbow/<element> at http://127.0.0.1:8708

In [49]:
open(vis, Window());
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8709
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://127.0.0.1:8709, 5)

In [35]:
function jacobian_transpose_ik!(state::MechanismState,
                               body::RigidBody,
                               point::Point3D,
                               desired::Point3D;
                               α=0.1,
                               iterations=100)
    mechanism = state.mechanism
    world = root_frame(mechanism)

    # Compute the joint path from world to our target body
    p = path(mechanism, root_body(mechanism), body)
    # Allocate the point jacobian (we'll update this in-place later)
    Jp = point_jacobian(state, p, transform(state, point, world))

    q = copy(configuration(state))

    for i in 1:iterations
        # Update the position of the point
        point_in_world = transform(state, point, world)
        # Update the point's jacobian
        point_jacobian!(Jp, state, p, point_in_world)
        # Compute an update in joint coordinates using the jacobian transpose
        Δq = α * Array(Jp)' * (transform(state, desired, world) - point_in_world).v
        # Apply the update
        q .= configuration(state) .+ Δq
        set_configuration!(state, q)
    end
    state
end

jacobian_transpose_ik! (generic function with 1 method)

In [36]:
rand!(state)
set_configuration!(vis, configuration(state))

In [37]:
desired_tip_location = Point3D(root_frame(mechanism), 0.5, 0, 2)

Point3D in "world": [0.5, 0.0, 2.0]

In [38]:
jacobian_transpose_ik!(state, body, point, desired_tip_location)
set_configuration!(vis, configuration(state))

In [39]:
transform(state, point, root_frame(mechanism))

Point3D in "world": [0.5000119896268409, 0.25, 2.002359458411753]

In [40]:
qs = typeof(configuration(state))[]

SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}[]

In [41]:
for x in range(-1, stop=1, length=100)
    desired = Point3D(root_frame(mechanism), x, 0, 2)
    jacobian_transpose_ik!(state, body, point, desired)
    push!(qs, copy(configuration(state)))
end
ts = collect(range(0, stop=1, length=length(qs)))
setanimation!(vis, Animation(vis, ts, qs))

In [42]:
circle_origin = SVector(0., 0.25, 2)
radius = 0.5
ω = 1.0  # radians per second at which the point should move in its circle

using GeometryTypes: Point

# Draw the circle in the viewer
θ = repeat(range(0, stop=2π, length=100), inner=(2,))[2:end]
cx, cy, cz = circle_origin
geometry = PointCloud(Point.(cx .+ radius .* sin.(θ), cy, cz .+ 0.5 .* cos.(θ)))
setobject!(vis[:circle], LineSegments(geometry, LineBasicMaterial()))

MeshCat Visualizer with path /meshcat/circle at http://127.0.0.1:8706

In [43]:
function make_circle_controller(state::MechanismState,
                                body::RigidBody,
                                point::Point3D,
                                circle_origin::AbstractVector,
                                radius,
                                ω)
    mechanism = state.mechanism
    world = root_frame(mechanism)
    joint_path = path(mechanism, root_body(mechanism), body)
    point_world = transform(state, point, root_frame(mechanism))
    Jp = point_jacobian(state, joint_path, point_world)
    v̇ = similar(velocity(state))

    function controller!(τ, t, state)
        desired = Point3D(world, circle_origin .+ radius .* SVector(sin(t / ω), 0, cos(t / ω)))
        point_in_world = transform_to_root(state, body) * point
        point_jacobian!(Jp, state, joint_path, point_in_world)
        Kp = 200
        Kd = 20
        Δp = desired - point_in_world
        v̇ .= Kp * Array(Jp)' * Δp.v .- 20 .* velocity(state)
        τ .= inverse_dynamics(state, v̇)
    end
end
controller! = make_circle_controller(state, body, point, circle_origin, radius, ω)
ts, qs, vs = simulate(state, 10, controller!);

In [44]:
setanimation!(vis, Animation(vis, ts, qs))